<a href="https://colab.research.google.com/github/cryoTUD/ColabScale/blob/development/ColabScale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://gitlab.tudelft.nl/aj-lab/locscale/-/raw/master/doc/img/LocScale_logo.png" height="200" align="right" style="height:240px">

#```ColabScale```

Easy to use cryo-EM map sharpening using [```LocScale```](https://gitlab.tudelft.nl/aj-lab/locscale) and generation of feature-enhanced maps with [```LocScale-EMmerNet```](https://gitlab.tudelft.nl/aj-lab/emmernet).


For more details, see <a href="#Instructions">instructions</a> at the bottom of the notebook and read our manuscripts.


In [1]:
# @title Setup environment
#@markdown #### Please make sure to connect to a GPU runtime before starting.
#%%capture
%%time
!pip install git+https://gitlab.tudelft.nl/aj-lab/locscale.git@development



  Cloning https://gitlab.tudelft.nl/aj-lab/locscale.git (to revision development) to /tmp/pip-req-build-tlceq4z3
  Running command git clone --filter=blob:none --quiet https://gitlab.tudelft.nl/aj-lab/locscale.git /tmp/pip-req-build-tlceq4z3
  Running command git checkout -b development --track origin/development
  Switched to a new branch 'development'
  Branch 'development' set up to track remote branch 'development' from 'origin'.
  Resolved https://gitlab.tudelft.nl/aj-lab/locscale.git to commit 147d06996d444a594f1822ff9c9ad458d5f929f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from locscale.automate.tools import LocScaleInputs, LocScaleRun

locscale_inputs = LocScaleInputs()
locscale_inputs.input["halfmap_paths"] = ["/content/emd_0193_half_map_1.map", "/content/emd_0193_half_map_2.map"]
locscale_inputs.input["output_path"] = "/content/emd_0193_map.mrc"
locscale_inputs.input["gpu_ids"] = ["0"]
locscale_inputs.input["batch_size"] = 16
locscale_inputs.input["number_processes"] = 4
locscale_inputs.input["verbose"] = True
locscale_run = LocScaleRun(locscale_inputs, "test", "model_free", 4)


In [ ]:
locscale_run.prepare_job()

In [ ]:
locscale_run.submit_job()

In [ ]:
from locscale.include.emmer.ndimage.map_utils import load_map
emmap_path = "/content/emd_0193_half_map_1.map"
emmap = load_map(emmap_path)[0]

In [ ]:
emmap.shape

(230, 230, 230)

In [1]:
#@markdown ### Input

from google.colab import files
from locscale.include.emmer.ndimage.map_tools import add_half_maps
import os
locscale_inputs = {}


job_name = 'test' #@param {type:"string"}
job_type = "model-free" # @param ["model-based", "model-free", "hybrid", "feature_enhance"]
#@markdown - __model-based__: ```LocScale``` sharpening using atomic model
#@markdown - __model-free__: ```LocScale``` sharpening without atomic model
#@markdown - __hybrid__: ```LocScale``` sharpening with partial atomic model
#@markdown - __feature_enhance__: Confidence-aware density modification with ```LocScale-EMmerNet```

upload_or_link = "upload" #@param ["upload", "link", "example"]

use_model = False #@param {type:"boolean"}
if use_model:
   if upload_or_link == "upload":
      input_model_path = os.path.join(job_name,f"model")
      os.makedirs(input_model_path, exist_ok=True)
      print("Please select model file...")
      uploaded = files.upload()
      input_model_path = os.path.join(input_model_path, list(uploaded.keys())[0])
      for model in uploaded.keys():
         if model.endswith('.pdb'):
            os.rename(model,os.path.join(input_model_path,model))
         else:
            print("Uploaded file is not a PDB file; please select correct file...")
            os.remove(model)
            uploaded = files.upload()
            os.rename(model,os.path.join(input_model_path,model))
            input_model_path = os.path.join(input_model_path, list(uploaded.keys())[0])
elif not use_model:
   input_model_path = None

use_mask = False #@param {type:"boolean"}
if use_mask:
   if upload_or_link == "upload":
      input_mask_path = os.path.join(job_name,f"mask")
      os.makedirs(input_mask_path, exist_ok=True)
      print("Please select mask file...")
      uploaded = files.upload()
      for mask in uploaded.keys():
         if mask.endswith('.mrc') or mask.endswith('.map'):
            input_mask_path = os.path.join(input_mask_path,mask)
            os.rename(mask, input_mask_path)
         else:
            print("Uploaded file is not a MRC file or MAP file; please select correct file...")

else:
   input_mask_path = None

#@markdown - `True` will request upload of atomic model as scaling reference. ADP refinement proceeds automaticallly.
#@markdown - `False` enables model-free scaling.

#input_model_path = "/content/map.mrc"

halfmap_paths = []
half_maps = {}

use_half_maps = True #@param {type:"boolean"}
if use_half_maps:
  input_half_map_path = os.path.join(job_name,f"half_maps")
  os.makedirs(input_half_map_path, exist_ok=True)
  print("Please select half maps...")
  uploaded = files.upload()
  for map in uploaded.keys():
     if map.endswith('.mrc') or map.endswith('.map'):
        new_map_path = os.path.join(input_half_map_path,map)
        halfmap_paths.append(new_map_path)
        os.rename(map,os.path.join(input_half_map_path,map))
     else:
        print("Uploaded file format is not either MRC or MAP; please select correct file...")
        os.remove(map)
        uploaded = files.upload()
        os.rename(model,os.path.join(input_half_map_path,map))

  apply_fsc_filter = False #@param {type:"boolean"}

elif not use_half_maps:
  uploaded = files.upload()
  if len(uploaded) == 1:
    input_map_dir = os.path.join(job_name,f"map")
    os.makedirs(input_map_dir, exist_ok=True)
    if map.endswith('.mrc') or map.endswith('.map'):
       input_map_path = os.path.join(input_map_dir, map)
       os.rename(map,os.path.join(input_map_dir,map))
    else:
       print("Uploaded file format is not either MRC or MAP; please select correct file...")
       os.remove(map)
       uploaded = files.upload()
       os.rename(model,os.path.join(input_map_dir,map))
       input_map_path = os.path.join(input_map_dir, map)

  else:
    print("Please select only one file...")



#@markdown - `True` requires upload of unfiltered half maps (__recommended__)
#@markdown - `False` will prompt upload of full map

#@markdown #### Other options

#model_resolution = None #@param {type:"string"}
symmetry = "C1" #@param {type:"string"}

#@markdown - Specifies point group for map symmetrisation. Supported groups are C<sub>n</sub>, D<sub>n</sub>, T, O, I
#@markdown - Helical symmetry is not yet supported

output_name = "emd_3061_mf.mrc " #@param {type:"string"}
#@markdown - Base string for output file names
#@markdown - `None` will use __`job_name`__

import gzip
from locscale.utils.file_tools import generate_filename_from_halfmap_path
def uncompress_if_needed(file_path):
  """Uncompresses the file at the given path if it is compressed."""
  if file_path.endswith('.gz'):
    uncompressed_path = file_path[:-3]
    with gzip.open(file_path, 'rb') as f_in:
      with open(uncompressed_path, 'wb') as f_out:
        f_out.write(f_in.read())
    print(f"Uncompressed {file_path} to {uncompressed_path}")
    return uncompressed_path
  return file_path

# Example usage for half maps:
if use_half_maps:
  input_half_map_path_1 = halfmap_paths[0]
  input_half_map_path_2 = halfmap_paths[1]
  input_half_map_path_1 = uncompress_if_needed(input_half_map_path_1)
  input_half_map_path_2 = uncompress_if_needed(input_half_map_path_2)
  emmap_path_filename = generate_filename_from_halfmap_path(input_half_map_path_1)
  emmap_path = add_half_maps(input_half_map_path_1, input_half_map_path_2, emmap_path_filename, fsc_filter=apply_fsc_filter)
else:
  input_map_path = uncompress_if_needed(input_map_path)
  emmap_path = input_map_path

# Example usage for model:
if use_model and upload_or_link == "upload":
  input_model_path = uncompress_if_needed(input_model_path)



from locscale.automate.tools import get_defaults_dictionary
locscale_inputs = get_defaults_dictionary()

locscale_inputs["emmap_path"] = emmap_path

locscale_inputs["outfile"] = output_name
locscale_inputs["symmetry"] = symmetry
locscale_inputs["mask"] = input_mask_path
locscale_inputs["model_coordinates"] = input_model_path




Please select half maps...


Saving emd_3061_half_map_1.map to emd_3061_half_map_1 (1).map
Saving emd_3061_half_map_2.map to emd_3061_half_map_2 (1).map


In [4]:
import multiprocessing
print(multiprocessing.cpu_count())

2


In [5]:
#@markdown ## Advanced Options

from google.colab import files
import os

#@markdown Most of these options should be left at default. Please only change if necessary and if you know what you are doing.

#@markdown #### FDR options

fdr_threshold = 0.01 #@param {type:"string"}
fdr_window_size = None #@param {type:"string"}
fdr_filter = None #@param {type:"string"}
averaging_filter_size = 3 #@param {type:"string"}
mask_threshold = 0.99 #@param {type:"string"}

locscale_inputs["fdr_threshold"] = fdr_threshold
locscale_inputs["fdr_window_size"] = fdr_window_size
locscale_inputs["fdr_filter"] = fdr_filter
locscale_inputs["averaging_filter_size"] = averaging_filter_size
locscale_inputs["mask_threshold"] = mask_threshold


#@markdown \

#@markdown #### EMmerNet options
low_context_model = False #@param {type:"boolean"}
batch_size = 16 #@param {type:"string"}
stride = 16 #@param {type:"string"}
gpu_ids = "0" #@param {type:"string"}

locscale_inputs["use_low_context_model"] = low_context_model
locscale_inputs["batch_size"] = int(batch_size)
locscale_inputs["gpu_ids"] = gpu_ids
locscale_inputs["stride"] = int(stride)

#@markdown \

#@markdown #### Reference options

model_resolution = None #@param {type:"string"}

#@markdown \

#@markdown #### Processing options
num_cpus = os.cpu_count()
number_processes = 2  #@param {type:"string"}
verbose = True #@param {type:"boolean"}

if int(number_processes) > num_cpus:
  number_processes = num_cpus

locscale_inputs["number_processes"] = int(number_processes)
locscale_inputs["verbose"] = verbose

In [8]:
halfmap_paths

['test/half_maps/emd_3061_half_map_1 (1).map',
 'test/half_maps/emd_3061_half_map_2 (1).map']

In [4]:
#@markdown ## Run LocScale
from locscale.utils.startup_utils import launch_locscale_no_mpi, launch_feature_enhance_no_mpi

import argparse
args = argparse.Namespace()
args.__dict__.update(locscale_inputs)

if job_type == "feature_enhance":
  launch_feature_enhance_no_mpi(args)
else:
  launch_locscale_no_mpi(args)

Running LocScale with modality: predict_model_map
................................................................................
Preparing your inputs for LocScale
All dependencies are satisfied. 

Adding the two half maps provided to generate a full map 

/content/test/half_maps/processing_files/emd_3061_half_map_1 (1).map
/content/test/half_maps/processing_files/emd_3061_half_map_2 (1).map
Saving as MRC file format with following properties: 
File name:  /content/test/half_maps/processing_files/EMD_3061_unsharpened_fullmap.mrc
Voxel size (1.4, 1.4, 1.4)
Origin (0., 0., 0.)
Shape (180, 180, 180)
Loading inputs... |
Done!
Calculating FDR confidence map... |
Done!
		Mask generated at /content/test/half_maps/processing_files/EMD_3061_unsharpened_fullmap_confidenceMap.mrc 

................................................................................
Preparing model map 


Emmernet model folder does not exist. Downloading model...

Model downloaded


Extracting: emmernet.tar.gz
Model

KeyboardInterrupt: 

In [ ]:
#@title #Analyse results
#@markdown ### Display scaled and unscaled maps

#!pip install stackview==0.8.0
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')
import stackview
import mrcfile
import ipywidgets as widgets
from skimage.io import imread
from google.colab import output
output.enable_custom_widget_manager()
from ipywidgets import HBox, VBox
#import pyclesperanto_prototype as cle

#cle.select_device("cupy")


export = False #@param {type:"boolean"}
#@markdown - export output as PNG images
input_colormap = "gray" #@param ['gray','plasma', 'viridis', 'inferno']
locscale_colormap = "inferno" #@param ['gray','plasma', 'viridis', 'inferno']
#@markdown - `gray`: greyscale
#@markdown - Other options are [perceptually uniform sequential color maps](https://matplotlib.org/stable/users/explain/colors/colormaps.html#sequential)
zoom_factor = 1 # @param {type:"number"}
display_style = "curtain" #@param {type:"string"}['curtain','stacked', 'toggle']

# load data
input_map = mrcfile.open('drive/MyDrive/ColabScale/input_map.mrc').data #needs to automatically load input map or averaged half maps
scaled_map = mrcfile.open('drive/MyDrive/ColabScale/scaled_map.mrc').data #needs to automatically load output map (locscale or feature_enhanced map)

# set scale
input_map = input_map/input_map.max()*255
scaled_map = scaled_map/scaled_map.max()*255

# set style & arrange widgets
if display_style == "curtain":
  print("Input map (left) vs. LocScale map (right)\n")
  w1 = stackview.curtain(input_map,scaled_map, zoom_factor=zoom_factor, axis=0, colormap=input_colormap, curtain_colormap=locscale_colormap)
  w2 = stackview.curtain(input_map,scaled_map, zoom_factor=zoom_factor, axis=1, colormap=input_colormap, curtain_colormap=locscale_colormap)
  w3 = stackview.curtain(input_map,scaled_map, zoom_factor=zoom_factor, axis=2, colormap=input_colormap, curtain_colormap=locscale_colormap)
  plot_map = HBox([w1, w2, w3])
elif display_style == "stacked":
  print("Input map (top) vs. LocScale map (bottom)\n")
  w1 = stackview.orthogonal(input_map,zoom_factor=zoom_factor, colormap=input_colormap)
  w2 = stackview.orthogonal(scaled_map,zoom_factor=zoom_factor, colormap=locscale_colormap)
  plot_map = VBox([w1, w2])
elif display_style == "toggle":
   print("Use buttons to toggle between maps")
   plot_map = stackview.switch(
     {"Input":    input_map,
     "LocScale": scaled_map,
     },
     colormap=[input_colormap, locscale_colormap],
     toggleable=True)
plot_map

Input map (left) vs. LocScale map (right)



In [ ]:
#@title #Analyse results
#@markdown ### Display scaled and unscaled maps

#!pip install stackview==0.8.0
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')
import stackview
import mrcfile
import ipywidgets as widgets
from skimage.io import imread
from google.colab import output
output.enable_custom_widget_manager()
from ipywidgets import HBox, VBox
#import pyclesperanto_prototype as cle

#cle.select_device("cupy")


export = False #@param {type:"boolean"}
#@markdown - export output as PNG images
input_colormap = "gray" #@param ['gray','plasma', 'viridis', 'inferno']
locscale_colormap = "inferno" #@param ['gray','plasma', 'viridis', 'inferno']
#@markdown - `gray`: greyscale
#@markdown - Other options are [perceptually uniform sequential color maps](https://matplotlib.org/stable/users/explain/colors/colormaps.html#sequential)
zoom_factor = 1 # @param {type:"number"}
display_style = "stacked" #@param {type:"string"}['curtain','stacked', 'toggle']

# load data
input_map = mrcfile.open('drive/MyDrive/ColabScale/emd3180/emd3180_half1.mrc').data #needs to automatically load input map or averaged half maps
scaled_map = mrcfile.open('drive/MyDrive/ColabScale/emd3180/emd3180_feature_enhanced.mrc').data #needs to automatically load output map (locscale or feature_enhanced map)

# set scale
input_map = input_map/input_map.max()*255
scaled_map = scaled_map/scaled_map.max()*255

# set style & arrange widgets
if display_style == "curtain":
  print("Input map (left) vs. LocScale map (right)\n")
  w1 = stackview.curtain(input_map,scaled_map, zoom_factor=zoom_factor, axis=0, colormap=input_colormap, curtain_colormap=locscale_colormap)
  w2 = stackview.curtain(input_map,scaled_map, zoom_factor=zoom_factor, axis=1, colormap=input_colormap, curtain_colormap=locscale_colormap)
  w3 = stackview.curtain(input_map,scaled_map, zoom_factor=zoom_factor, axis=2, colormap=input_colormap, curtain_colormap=locscale_colormap)
  plot_map = HBox([w1, w2, w3])
elif display_style == "stacked":
  print("Input map (top) vs. LocScale map (bottom)\n")
  w1 = stackview.orthogonal(input_map,zoom_factor=zoom_factor, colormap=input_colormap)
  w2 = stackview.orthogonal(scaled_map,zoom_factor=zoom_factor, colormap=locscale_colormap)
  plot_map = VBox([w1, w2])
elif display_style == "toggle":
   print("Use buttons to toggle between maps")
   plot_map = stackview.switch(
     {"Input":    input_map,
     "LocScale": scaled_map,
     },
     colormap=[input_colormap, locscale_colormap],
     toggleable=True)
plot_map

Input map (top) vs. LocScale map (bottom)



In [ ]:
#!pip install ipyvolume
#import ipyvolume
#!jupyter nbextension enable /usr/local/lib/python3.8/site-packages/ipyvolume
#from google.colab import output
#output.enable_custom_widget_manager()
#ipyvolume.volshow(input_map)
import sys
print(sys.prefix)

/usr


In [ ]:
#@title Package and download results
#@markdown If you are having issues downloading the result archive, try disabling your adblocker and run this cell again. If that fails click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\".


files.download(f"{jobname}.result.zip")

if save_to_google_drive == True and drive:
  uploaded = drive.CreateFile({'title': f"{jobname}.result.zip"})
  uploaded.SetContentFile(f"{jobname}.result.zip")
  uploaded.Upload()
  print(f"Uploaded {jobname}.result.zip to Google Drive with ID {uploaded.get('id')}")